Libraries

In [1]:
import numpy as np
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import os


In [2]:
csv1 = pd.read_csv('degradation_data\S1.csv')
csv2 = pd.read_csv('degradation_data\S2.csv')
csv3 = pd.read_csv('degradation_data\S3.csv')
csv4 = pd.read_csv('degradation_data\S4.csv')
csv5 = pd.read_csv('degradation_data\S5.csv')

print(csv1)

    time    test_0    test_1    test_2    test_3    test_4    test_5  \
0      0  1.017641  1.016243  0.995832  1.017886  1.000506  1.004412   
1    120  0.999501  1.001721  0.981776  0.929673  0.986303  0.990320   
2    168  0.992187  0.995876  0.976231  0.899056  0.980551  0.984723   
3    216  0.984921  0.990061  0.970740  0.870532  0.974819  0.979171   
4    288  0.974139  0.981413  0.962607  0.831181  0.966282  0.970936   
5    336  0.967040  0.975705  0.957254  0.806971  0.960640  0.965510   
6    384  0.960016  0.970046  0.951957  0.784202  0.955041  0.960136   
7    456  0.949628  0.961651  0.944113  0.752470  0.946728  0.952174   
8    552  0.936057  0.950640  0.933843  0.714122  0.935813  0.941741   
9    624  0.926092  0.942520  0.926280  0.687940  0.927755  0.934052   
10   672  0.919549  0.937172  0.921303  0.671572  0.922446  0.928991   
11   720  0.913087  0.931878  0.916377  0.656000  0.917186  0.923981   
12   792  0.903542  0.924034  0.909083  0.634007  0.909391  0.91

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\eusta\AppData\Local\Temp\ipykernel_11452\3958311247.py:1: SyntaxWarning: invalid escape sequence '\S'
  csv1 = pd.read_csv('degradation_data\S1.csv')
C:\Users\eusta\AppData\Local\Temp\ipykernel_11452\3958311247.py:2: SyntaxWarning: invalid escape sequence '\S'
  csv2 = pd.read_csv('degradation_data\S2.csv')
C:\Users\eusta\AppData\Local\Temp\ipykernel_11452\3958311247.py:3: SyntaxWarning: invalid escape sequence '\S'
  csv3 = pd.read_csv('degradation_data\S3.csv')
C:\Users\eusta\AppData\

In [3]:
time = csv1['time'].tolist()
test_0 = csv1['test_0'].tolist()

time = np.array(time)
test_0 = np.array(test_0)

In [4]:
# --- 1. Régression Linéaire ---
# Appliquer la régression linéaire
model_lin = LinearRegression()
T_reshaped = time.reshape(-1, 1)  # Maintenant T est un numpy array, donc reshape fonctionne
model_lin.fit(T_reshaped, test_0)
pred_lin = model_lin.predict(T_reshaped)
mse_lin = mean_squared_error(test_0, pred_lin)

# --- 2. Modèle Exponentiel ---
# Définir la fonction exponentielle : y = a * exp(b * x)
def exp_model(x, a, b):
    return a * np.exp(b * x)

# Initialisation des paramètres pour exp_model
# Essayer une valeur initiale raisonnable pour a et b (ici, a=1, b=0.1)
params_initial = [1, 0.1]

# Ajuster les paramètres du modèle exponentiel
try:
    params_exp, _ = curve_fit(exp_model, time, test_0, p0=params_initial, maxfev=10000)
    pred_exp = exp_model(time, *params_exp)
    mse_exp = mean_squared_error(test_0, pred_exp)
except RuntimeError as e:
    print("Erreur d'ajustement pour le modèle exponentiel:", e)
    params_exp = None
    mse_exp = np.inf

# --- 3. Modèle Logarithmique ---
# Définir la fonction logarithmique : y = a * log(b * x)
def log_model(x, a, b):
    return a * np.log(b * x + 1)  # Ajout de 1 pour éviter log(0)

# Ajuster les paramètres du modèle logarithmique
params_log, _ = curve_fit(log_model, time, test_0)
pred_log = log_model(time, *params_log)
mse_log = mean_squared_error(test_0, pred_log)

# --- 4. Affichage des résultats ---
plt.figure(figsize=(10, 6))
plt.plot(time, test_0, 'o', label="Données réelles", color='black')

# Régression linéaire
plt.plot(time, pred_lin, label=f"Régression linéaire (MSE: {mse_lin:.2f})", color='red')

# Modèle exponentiel si ajustement réussi
if params_exp is not None:
    plt.plot(time, pred_exp, label=f"Modèle exponentiel (MSE: {mse_exp:.2f})", color='green')

# Modèle logarithmique
plt.plot(time, pred_log, label=f"Modèle logarithmique (MSE: {mse_log:.2f})", color='blue')

plt.xlabel('Temps (T)')
plt.ylabel('Valeur (X)')
plt.title('Ajustement des modèles aux données')
plt.legend()
plt.show()

# --- 5. Sélection du meilleur modèle ---
# Comparer les MSE (Mean Squared Error) pour choisir le meilleur modèle
errors = {'Linéaire': mse_lin, 'Exponentiel': mse_exp if params_exp is not None else np.inf, 'Logarithmique': mse_log}
best_model = min(errors, key=errors.get)
print(f"Le meilleur modèle est : {best_model} avec une erreur MSE de {errors[best_model]:.2f}")

Unexpected exception formatting exception. Falling back to standard exception


C:\Users\eusta\AppData\Local\Temp\ipykernel_11452\4158331840.py:31: RuntimeWarning: invalid value encountered in log
  return a * np.log(b * x + 1)  # Ajout de 1 pour éviter log(0)
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\eusta\AppData\Local\Temp\ipykernel_11452\4158331840.py", line 34, in <module>
    params_log, _ = curve_fit(log_model, time, test_0)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\scipy\optimize\_minpack_py.py", line 1005, in curve_fit
    raise RuntimeError("Optimal parameters not found: " + errmsg)
RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 600.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\sit